# Desafio de Previsão de Sucesso de Startups
## Objetivo do Projeto

O objetivo deste projeto é desenvolver um modelo de machine learning capaz de prever o sucesso de startups com base em dados históricos. O sucesso é definido como uma startup que permanece ativa ou foi adquirida (label = 1), enquanto o insucesso é uma startup que fechou (label = 0).

O modelo apoiará investidores e aceleradoras na tomada de decisões estratégicas, otimizando investimentos e maximizando o impacto econômico. A métrica principal para avaliação do modelo é a Acurácia.

### Estrutura do Notebook:

O processo será dividido nas seguintes etapas:

- Configuração Inicial: Carregamento das bibliotecas e dos dados.

- Análise Exploratória de Dados: Entendimento da estrutura, estatísticas e distribuição dos dados, além da formulação e validação de hipóteses.

- Pré-processamento e Engenharia de Features: Preparação dos dados para a modelagem, incluindo a criação de novas variáveis.

- Construção e Otimização do Modelo: Definição do pipeline, treinamento, otimização de hiperparâmetros e avaliação do modelo final.

- Análise de Importância das Features: Verificação de quais variáveis foram mais influentes para o modelo.

- Submissão: Geração do arquivo de resultados para o conjunto de teste.



## 1. Imports e Configurações

Nesta primeira etapa, todas as bibliotecas necessárias são importadas.

In [ ]:
# importanto todas as bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# para gráficos interativos no Windows
matplotlib.use('TkAgg')

## 2. Carregamento dos dados

Agora, para a análise, são carregados os conjuntos de dados de treino e teste.

In [17]:
# Verifique se o seu código está lendo os arquivos originais
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

## 3. Preparação dos dados

Antes de construir o modelo, é fundamental explorar os dados para entender suas características, identificar padrões, verificar a necessidade de tratamento e formular hipóteses.

In [18]:
# separar features e target
X = train_df.drop(columns=['id', 'labels'])
y = train_df['labels']
X_test = test_df.drop(columns=['id'])

# identificar colunas
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = ['category_code']

## 4. Análise Descritiva do Dataset

### Principais Insights:

- O dataset de treino possui 646 registros e 33 colunas.

- Existem valores nulos (NaN) em colunas importantes como age_first_funding_year e age_first_milestone_year, que precisarão ser tratados.

- A variável alvo 

- labels está moderadamente desbalanceada, com aproximadamente 65% de casos de sucesso (1) e 35% de casos de insucesso (0).

In [19]:
# informações básicas do dataset
print("Informações Gerais do DataFrame de Treino:")
train_df.info()

print("\nEstatísticas Descritivas das Colunas Numéricas:")
display(train_df.describe())

print("\nVerificando a Proporção da Variável Alvo (labels):")
print(train_df['labels'].value_counts(normalize=True))

Informações Gerais do DataFrame de Treino:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        646 non-null    int64  
 1   age_first_funding_year    611 non-null    float64
 2   age_last_funding_year     637 non-null    float64
 3   age_first_milestone_year  508 non-null    float64
 4   age_last_milestone_year   535 non-null    float64
 5   relationships             646 non-null    int64  
 6   funding_rounds            646 non-null    int64  
 7   funding_total_usd         646 non-null    int64  
 8   milestones                646 non-null    int64  
 9   is_CA                     646 non-null    int64  
 10  is_NY                     646 non-null    int64  
 11  is_MA                     646 non-null    int64  
 12  is_TX                     646 non-null    int64  
 13  is_otherstate         

,id,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,labels
count,646.000000,611.000000,637.000000,508.000000,535.000000,646.000000,646.000000,6.460000e+02,646.000000,646.000000,...,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000
mean,461.577399,2.341718,4.037724,3.352657,4.944729,7.948916,2.351393,2.949633e+07,1.913313,0.546440,...,0.003096,0.304954,0.329721,0.260062,0.515480,0.419505,0.235294,0.091331,2.848655,0.647059
std,264.859464,2.468275,2.950923,2.866952,3.213319,7.397602,1.357856,2.261999e+08,1.337095,0.498224,...,0.055598,0.460745,0.470476,0.439008,0.500148,0.493860,0.424511,0.288303,1.894050,0.478255
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.100000e+04,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,233.250000,0.680000,1.870000,1.185000,2.540000,3.000000,1.000000,3.000000e+06,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,0.000000
50%,459.500000,1.650000,3.610000,2.785000,4.620000,6.000000,2.000000,1.020000e+07,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.333300,1.000000
75%,692.500000,3.600000,5.590000,4.935000,6.880000,10.000000,3.000000,2.587500e+07,3.000000,1.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,1.000000
max,923.000000,21.900000,21.900000,24.680000,24.680000,63.000000,8.000000,5.700000e+09,6.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000,1.000000



Verificando a Proporção da Variável Alvo (labels):
labels
1    0.647059
0    0.352941
Name: proportion, dtype: float64


## 5. Análise Visual e Validação de Hipóteses

Aqui, serão utilizados gráficos para investigar as hipóteses e entender melhor a distribuição de algumas variáveis-chave.

### Formulação de Hipóteses
Com base na descrição do problema e na análise inicial, seguintes hipóteses foram formuladas:

- Hipótese 1 (Localização Estratégica): Startups localizadas na Califórnia (is_CA = 1) possuem uma taxa de sucesso maior, devido ao acesso facilitado a investimentos e talentos do Vale do Silício.

- Hipótese 2 (Capital Social): O número de conexões estratégicas (relationships) é um forte indicador de sucesso, pois uma rede maior facilita o acesso a recursos e oportunidades.

- Hipótese 3 (Validação de Mercado): Um alto volume de investimento total (funding_total_usd) está positivamente correlacionado com o sucesso, pois indica confiança do mercado no potencial da startup.

In [20]:
# gráficos para análise
# investigando a hipótese 1
sns.barplot(x='is_CA', y='labels', data=train_df, errorbar=None)
plt.title('Taxa de Sucesso: Califórnia vs. Outros Estados')
plt.ylabel('Taxa Média de Sucesso')
plt.show()

# investigando a hipótese 2
sns.boxplot(x='labels', y='relationships', data=train_df)
plt.title('Distribuição de Conexões por Sucesso da Startup')
plt.xlabel('0 = Insucesso, 1 = Sucesso')
plt.show()

# análise da principal variável categórica
plt.figure(figsize=(12, 6))
sns.countplot(y='category_code', data=train_df, order=train_df['category_code'].value_counts().index)
plt.title('Distribuição de Startups por Setor')
plt.xscale('log') # usar escala log para melhor visualização se houver muita variação
plt.show()

### Interpretação dos Gráficos:

- O primeiro gráfico mostra que startups da Califórnia (is_CA = 1) de fato apresentam uma taxa de sucesso ligeiramente superior, apoiando a Hipótese 1.

- O boxplot indica que startups de sucesso (labels = 1) tendem a ter um número maior de conexões (relationships), validando a Hipótese 2.

- O último gráfico revela que os setores de software, web, mobile e enterprise são os mais comuns no dataset.

## 6. Otimização do Modelo

Com a preparação de dados estruturada, o próximo passo é ir para a modelagem. Após testes iniciais, o GradientBoostingClassifier foi escolhido por seu alto potencial de performance. Agora, basta otimizar seus hiperparâmetros para encontrar a melhor configuração possível.

In [21]:
# carregar e separar os dados
X = train_df.drop(columns=['id', 'labels'])
y = train_df['labels']

# função para criar novas features
def create_features(df):
    df_transformed = df.copy()
    df_transformed['funding_duration_years'] = df_transformed['age_last_funding_year'] - df_transformed['age_first_funding_year']
    df_transformed.loc[df_transformed['funding_duration_years'] < 0, 'funding_duration_years'] = 0
    df_transformed['milestone_duration_years'] = df_transformed['age_last_milestone_year'] - df_transformed['age_first_milestone_year']
    df_transformed.loc[df_transformed['milestone_duration_years'] < 0, 'milestone_duration_years'] = 0
    df_transformed['usd_per_round'] = df_transformed['funding_total_usd'] / (df_transformed['funding_rounds'] + 1e-6)
    return df_transformed

# pipeline de pré-processamento
new_numerical_features = X.select_dtypes(include=np.number).columns.tolist() + [
    'funding_duration_years', 'milestone_duration_years', 'usd_per_round'
]
categorical_features = ['category_code']
numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, new_numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

# pipeline final com o novo modelo
# usar o GradientBoostingClassifier no pipeline
pipeline_final = Pipeline(steps=[
    ('feature_creation', FunctionTransformer(create_features)),
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42)) # Removido class_weight, não é um parâmetro direto aqui
])

# otimização com um novo grid de parâmetros, específico para o Gradient Boosting
param_dist_gb = {
    "classifier__n_estimators": [100, 200, 300],
    "classifier__learning_rate": [0.01, 0.05, 0.1], # taxa de aprendizado, o parâmetro mais importante
    "classifier__max_depth": [3, 5, 7],            # profundidade das árvores
    "classifier__subsample": [0.8, 0.9, 1.0],      # fração de amostras para treinar cada árvore
    "classifier__max_features": ["sqrt", "log2"]
}
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(
    estimator=pipeline_final,
    param_distributions=param_dist_gb,
    n_iter=50, # manter uma busca ampla
    cv=cv_strategy,
    scoring="accuracy",
    random_state=42,
    verbose=1,
    n_jobs=-1
)

print("Iniciando a otimização com Gradient Boosting e Engenharia de Features...")
random_search.fit(X, y)

# exibir resultados finais
print("\nResultados da Otimização Final:")
print(f"Melhor acurácia encontrada: {random_search.best_score_:.4f}")
print("\nMelhores hiperparâmetros:")
print(random_search.best_params_)

print("\nAvaliando o Desempenho do Modelo Otimizado Definitivo...")
best_model_gb = random_search.best_estimator_
scoring_metrics = ['accuracy', 'f1', 'roc_auc']
final_cv_results = cross_validate(best_model_gb, X, y, cv=cv_strategy, scoring=scoring_metrics)

print("\nNovas Métricas Médias (Gradient Boosting Otimizado):")
print(f"  - Acurácia: {np.mean(final_cv_results['test_accuracy']):.4f}")
print(f"  - F1-score: {np.mean(final_cv_results['test_f1']):.4f}")
print(f"  - ROC-AUC : {np.mean(final_cv_results['test_roc_auc']):.4f}")

Iniciando a otimização com Gradient Boosting e Engenharia de Features...
Fitting 5 folds for each of 50 candidates, totalling 250 fits

Resultados da Otimização Final:
Melhor acurácia encontrada: 0.8003

Melhores hiperparâmetros:
{'classifier__subsample': 0.8, 'classifier__n_estimators': 300, 'classifier__max_features': 'log2', 'classifier__max_depth': 3, 'classifier__learning_rate': 0.05}

Avaliando o Desempenho do Modelo Otimizado Definitivo...

Novas Métricas Médias (Gradient Boosting Otimizado):
  - Acurácia: 0.8003
  - F1-score: 0.8539
  - ROC-AUC : 0.8081


### Resultado da Modelagem:

O modelo final, após a engenharia de features e a otimização de hiperparâmetros, atingiu uma acurácia média superior a 80%, cumprindo o requisito principal do desafio. As métricas de F1-score e ROC-AUC também indicam um modelo robusto e com bom poder de discriminação.

## 7. Análise de Importância das Features

Para gerar mais insights de negócio, é necessário inspecionar o modelo final e descobrir quais variáveis ele considerou mais importantes para fazer suas previsões.

In [27]:
print("Análise de Importância das Features do Modelo Final:")

try:
    # obter os nomes das features após o pré-processamento no pipeline
    # o pipeline já foi treinado, então pode extrair as informações dele
    
    # nomes das features numéricas
    num_features = best_model_gb.named_steps['preprocessor'].transformers_[0][2]
    
    # nomes das features categóricas transformadas pelo OneHotEncoder
    ohe_features = best_model_gb.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
    
    # juntar todos os nomes na ordem correta
    all_feature_names = num_features + list(ohe_features)

    # obter os valores de importância do classificador treinado
    importances = best_model_gb.named_steps['classifier'].feature_importances_

    # criar um DataFrame para facilitar a visualização
    feature_importance_df = pd.DataFrame({
        'feature': all_feature_names,
        'importance': importances
    }).sort_values(by='importance', ascending=False).head(15) # Pegando as 15 mais importantes

    # plotar o gráfico
    plt.figure(figsize=(10, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance_df, palette='rocket')
    plt.title('Top 15 Features Mais Importantes para o Modelo')
    plt.xlabel('Importância (calculada pelo Gradient Boosting)')
    plt.ylabel('Feature')
    plt.show()

except Exception as e:
    print(f"\nNão foi possível gerar o gráfico de importância de features. Erro: {e}")
    print("Isso pode acontecer se os nomes das features no pipeline não foram extraídos corretamente.")

Análise de Importância das Features do Modelo Final:


C:\Users\sofia\AppData\Local\Temp\ipykernel_31832\1571294054.py:27: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='importance', y='feature', data=feature_importance_df, palette='rocket')


### Insights:

A análise de importância das features revela quais fatores são mais decisivos para o sucesso de uma startup, segundo o modelo. Variáveis como o total de investimento (funding_total_usd), a idade no último investimento (age_last_funding_year) e o número de conexões (relationships) geralmente aparecem no topo, fornecendo insights valiosos para a aceleradora.

## 8. Predição e Submissão

Por último, o modelo treinado será utilizado e otimizado para fazer as previsões no conjunto de teste e gerar o arquivo submission.csv no formato exigido.

In [25]:
print("Fazendo previsões no conjunto de teste com o modelo otimizado...")

X_test_final = test_df.drop(columns=['id'])
test_predictions_final = best_model_gb.predict(X_test_final)

# criar o DataFrame de submissão
submission_df_final = pd.DataFrame({
    'id': test_df['id'],
    'labels': test_predictions_final
})

# salvar o arquivo CSV
submission_path_final = 'submission_final.csv'
submission_df_final.to_csv(submission_path_final, index=False)

print(f"\nArquivo de submissão '{submission_path_final}' criado com sucesso!")
display(submission_df_final.head())

Fazendo previsões no conjunto de teste com o modelo otimizado...

Arquivo de submissão 'submission_final.csv' criado com sucesso!


,id,labels
0,70,1
1,23,0
2,389,1
3,872,1
4,920,1


## 9. Conclusão

Este projeto demonstrou um fluxo completo de machine learning para prever o sucesso de startups. Partindo de uma análise exploratória, hipóteses foram desenvolvidas e validadas com visualizações. Em seguida, através da engenharia de features e da otimização de um modelo de Gradient Boosting, foi construído um classificador com performance superior a 80% de acurácia, atendendo ao objetivo principal do desafio.

O modelo final e os insights sobre a importância das features podem agora ser utilizados pela aceleradora para embasar suas decisões de investimento de forma mais estratégica e orientada por dados.